# Импорт библиотек

In [2]:
import pandas as pd
import json
import os

# Чтение данных

In [3]:
all_data = []
# Перебираем все файлы в директории
for filename in os.listdir('data'):
    if filename.endswith('.json'): # Проверяем, что файл имеет расширение .json
        file_path = os.path.join('data', filename)
        # Открываем
        with open(file_path, 'r', encoding='utf-8') as file:
            data = [json.loads(line) for line in file]
            all_data.extend(data)

In [4]:
data_df = pd.DataFrame(all_data)
# Так как в данных есть списки со словарями, преобразуем их просто в словари, чтобы можно было преобразить данные в DataFrame без словарей:
# Ключи словарей станут новыми признаками в DataFrame 
def convert_list_to_dict(cell):
    if isinstance(cell, list) and len(cell) > 0:
        return cell[0]
    else:
        return cell

# Применение функции к каждой ячейке в каждом столбце DataFrame
for column in data_df.columns:
    data_df[column] = data_df[column].apply(convert_list_to_dict)

In [5]:
# Преобразуем данные в json, чтобы открыть через json_normalize, сразу добавив ключи, как признаки
data_df_1 = pd.DataFrame(data_df)

clear_json = data_df_1.to_json(orient='records')

data_list = json.loads(clear_json)

full_data = pd.json_normalize(data_list)
# отсортируем признаки по алфавиту
full_data = full_data.sort_index(axis=1)

In [6]:
# Запишем в  готовую таблицу в csv файл
full_data.to_csv('data/Полная таблица.csv')


# Знакомство с данными

In [7]:
df = pd.read_csv('data/Полная таблица.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

C:\Users\Aplatonov\AppData\Local\Temp\ipykernel_3044\2871854370.py:1: DtypeWarning: Columns (171,175,177,179,182,201,202,203,204,205,206,211,214,215) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Полная таблица.csv')


,addOkved,addOkved.code,addOkved.name,addOkved.version,address,administrators,administrators.name,administrators.title,charter,dateLiquid,...,socialMedia.youtube,statusDetail.code,statusDetail.name,statusDetail.shortName,successors,successors.fullName,successors.inn,successors.ogrn,website,website_punycode
0,[],NaN,NaN,NaN,"191186, ГОРОД САНКТ-ПЕТЕРБУРГ, УЛИЦА ИТАЛЬЯНСК...",NaN,КОШЕЧКИН ДЕНИС ВИКТОРОВИЧ,ПРЕДСЕДАТЕЛЬ ПРАВЛЕНИЯ,NaN,NaN,...,NaN,NaN,NaN,Действующая,[],NaN,NaN,NaN,NaN,NaN
1,NaN,58.11,Издание книг,ОК 029-2014 (КДЕС Ред. 2),"111395, ГОРОД МОСКВА, УЛИЦА МОЛДАГУЛОВОЙ, ДОМ ...",NaN,НИЗАМИЕВ РУСЛАН ВИЛЬДАНОВИЧ,ГЕНЕРАЛЬНЫЙ ДИРЕКТОР,NaN,NaN,...,NaN,NaN,NaN,Действующая,[],NaN,NaN,NaN,NaN,NaN
2,[],NaN,NaN,NaN,"197198, ГОРОД САНКТ-ПЕТЕРБУРГ, УЛИЦА БОЛЬШАЯ П...",NaN,САМОЙЛОВА АНЖЕЛА ВАСИЛЬЕВНА,ПРЕДСЕДАТЕЛЬ КОЛЛЕГИИ,NaN,NaN,...,NaN,NaN,NaN,Действующая,[],NaN,NaN,NaN,NaN,NaN
3,NaN,68.20,Аренда и управление собственным или арендованн...,ОК 029-2014 (КДЕС Ред. 2),"125424, ГОРОД МОСКВА, ШОССЕ ВОЛОКОЛАМСКОЕ, ДОМ...",NaN,ГЕЛЬДТ ВЛАДИМИР АНДРЕЕВИЧ,ПРЕДСЕДАТЕЛЬ ТОВАРИЩЕСТВА,NaN,NaN,...,NaN,NaN,NaN,Действующая,[],NaN,NaN,NaN,NaN,NaN
4,NaN,73.20,Исследование конъюнктуры рынка и изучение обще...,ОК 029-2014 (КДЕС Ред. 2),"197198, ГОРОД САНКТ-ПЕТЕРБУРГ, УЛИЦА ЗВЕРИНСКА...",NaN,ПРИВАЛОВА ЮЛИЯ АНАТОЛЬЕВНА,ДИРЕКТОР,NaN,NaN,...,NaN,NaN,NaN,Действующая,[],NaN,NaN,NaN,NaN,NaN


In [8]:
print(df.info(verbose=True))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747034 entries, 0 to 747033
Data columns (total 215 columns):
 #    Column                                Dtype  
---   ------                                -----  
 0    addOkved                              object 
 1    addOkved.code                         object 
 2    addOkved.name                         object 
 3    addOkved.version                      object 
 4    address                               object 
 5    administrators                        object 
 6    administrators.name                   object 
 7    administrators.title                  object 
 8    charter                               float64
 9    dateLiquid                            float64
 10   dateLiquid.$date                      object 
 11   dateOgrn.$date                        object 
 12   dateReg                               float64
 13   dateReg.$date                         object 
 14   egrulStatus                           object 
 15 

In [9]:
missing_percentage = df.isnull().mean() * 100
print(f'В данных {missing_percentage[missing_percentage == 0].count()} признаков без записей (только пропуски) и {missing_percentage[(missing_percentage != 0) & (missing_percentage <= 10)].count()} признаков, где до 10% записей')
# Признаки только с пропусками можно удалить
df_no_nan = df.dropna(axis=1, how='all')

В данных 21 признаков без записей (только пропуски) и 10 признаков, где до 10% записей


# Очистка данных

In [10]:
# Просмотрим все признаки типа object
object_columns = df.select_dtypes(include=['object'])
print(len(object_columns.columns), 'столбцов object')

47 столбцов object


In [11]:
df.shape

(747034, 215)

In [12]:
lest_obj = []
for col in object_columns.columns:
    if df[col].nunique() == 1: # выберем столбцы, в которых одина уникальная запись
        lest_obj.append(col)
        print(df[col].value_counts())

addOkved
[]    474366
Name: count, dtype: int64
administrators
[]    48722
Name: count, dtype: int64
predecessors
[]    735550
Name: count, dtype: int64
reports
[]    614158
Name: count, dtype: int64
socialMedia.instagram
https://www.instagram.com/crimea24news    1
Name: count, dtype: int64
successors
[]    732281
Name: count, dtype: int64


In [13]:
# Так как это просто пустые списки и одна ссылка на соцсеть, можно удалить эти признаки
df_no_nan = df.drop(columns=lest_obj)

In [14]:
# Также удалим признаки, где 95% пропусков
df_no_nan = df_no_nan.dropna(axis=1, thresh=int(len(df_no_nan) * 0.05))


In [15]:
df_no_nan.shape
# Количество кризнаков сократилось в четыре раза 

(747034, 54)

In [17]:
df_no_nan.to_csv('data/Полная таблица очищенная.csv')

In [18]:
df_full = pd.read_csv('data/Полная таблица очищенная.csv')
df_full = df_full.copy()

C:\Users\Aplatonov\AppData\Local\Temp\ipykernel_3044\3928868559.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv('data/Полная таблица очищенная.csv')


### Проанализируем признаки

In [19]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747034 entries, 0 to 747033
Data columns (total 55 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Unnamed: 0                            747034 non-null  int64  
 1   addOkved.code                         272668 non-null  object 
 2   addOkved.name                         272668 non-null  object 
 3   addOkved.version                      272668 non-null  object 
 4   address                               747034 non-null  object 
 5   administrators.name                   698312 non-null  object 
 6   administrators.title                  698312 non-null  object 
 7   dateLiquid.$date                      399961 non-null  object 
 8   dateOgrn.$date                        747034 non-null  object 
 9   dateReg.$date                         321416 non-null  object 
 10  egrulStatus                           747034 non-null  object 
 11  

- От дополнительного ОКВЭД вдяд ли есть зависимость одобрения гранта - удаляем
- Точный адрес тоже не должен влиять (посммотрим позже на регион и город) - удаляем
- Имя руководителя как и должность тоже ничего не должно значить - удаляем
- ИНН и КПП не влияют - удаляем
- Код основного ОКВЭД и  его версия ничего нам не скажет (оставиим название) - удаляем
- Реестровый номер НКО в системе Минюста РФ - удаляем
- ОГРН - удаляем
- код ОКАТО точно не нужен, оставим расшифровку
- код ОКФС - удаляем
- код ОКОГУ - удаляем
- код ОКТМО - удаляем
- код ОПФ и версия- удаляем
- ID отчета и тип - удаляем

- Статус НКО в системе Минюста РФ не должен влиять на одобрение гранта, но пока оставим

Остальные признаки будем обрабатывать.   
Возможно, ещё что-то удалим в процессе

hasRegionalSupport - целевой признак одобрения получения региональной поддержки

In [21]:
del_list = [
    'addOkved.code',
    'addOkved.version',
    'address',
    'administrators.name',
    'administrators.title',
    'inn',
    'kpp',
    'mainOkved.code',
    'mainOkved.version',
    'ogrn',
    'okato.code',
    'okfs.code',
    'okogu.code',
    'oktmo.code',
    'opf.code',
    'opf.version',
    'reports.reportId',
    'reports.type'
]
df_clean = df_full.drop(columns=del_list)

In [22]:
df_clean

,Unnamed: 0,addOkved.name,dateLiquid.$date,dateOgrn.$date,dateReg.$date,egrulStatus,fullName,hasRegionalSupport,incomeDetail.contracts223.totalCount,incomeDetail.contracts223.totalSum,...,oktmo.name,opf.name,originDate.$date,regionCode,regionName,reports.localUrl,reports.minjustUrl,reports.publishYear,shortName,statusDetail.shortName
0,0,NaN,NaN,2020-10-30T00:00:00.000Z,NaN,Действует,МЕЖРЕГИОНАЛЬНАЯ АНТРОПОСОФСКАЯ МЕДИЦИНСКАЯ АСС...,False,0,0.0,...,NaN,Ассоциации (союзы),2020-10-30T00:00:00.000Z,78,Санкт-Петербург,NaN,NaN,NaN,"""АМА""",Действующая
1,1,Издание книг,NaN,2020-09-23T00:00:00.000Z,NaN,Действует,"ФОНД ""СОДЕЙСТВИЕ ДУХОВНОМУ РАЗВИТИЮ ЧЕЛОВЕКА ""...",False,0,0.0,...,NaN,Фонды,2020-09-23T00:00:00.000Z,77,Москва,"http://cdnx.h5.crftr.net/f/4,18cff2e40258fa.pdf",http://unro.minjust.ru/NKOReports.aspx?mode=sh...,2021.0,"ФОНД ""УНИВЕРСАЛИСТ М""",Действующая
2,2,NaN,NaN,2020-02-03T00:00:00.000Z,NaN,Действует,"КОЛЛЕГИЯ АДВОКАТОВ ""САНКТ-ПЕТЕРБУРГСКАЯ КОЛЛЕГ...",False,0,0.0,...,NaN,Коллегии адвокатов,2020-02-03T00:00:00.000Z,78,Санкт-Петербург,NaN,NaN,NaN,NaN,Действующая
3,3,Аренда и управление собственным или арендованн...,NaN,2020-09-23T00:00:00.000Z,NaN,Действует,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ НЕДВИЖИМОСТИ ""ЛУГИН...",False,0,0.0,...,NaN,Товарищества собственников недвижимости,2020-09-23T00:00:00.000Z,77,Москва,NaN,NaN,NaN,"ТСН ""ЛУГИНИНО ПАРК""",Действующая
4,4,Исследование конъюнктуры рынка и изучение обще...,NaN,2020-07-08T00:00:00.000Z,NaN,Действует,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ЦЕНТР С...",False,0,0.0,...,NaN,Автономные некоммерческие организации,2020-07-08T00:00:00.000Z,78,Санкт-Петербург,NaN,NaN,NaN,"АНО ""ЦЕНТР ""ПЕТЕРБУРГФОРУМ""",Действующая
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747029,747029,Управление эксплуатацией жилого фонда,2009-01-12T00:00:00.000Z,2006-03-29T00:00:00.000Z,NaN,Ликвидирована,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ ЖИЛЬЯ ""БОЛЬШОЙ ХАРИ...",False,0,0.0,...,NaN,Товарищество собственников жилья,2006-03-29T00:00:00.000Z,77,Москва,NaN,NaN,NaN,"ТСЖ ""Б. ХАРИТОНЬЕВСКИЙ, 9""",Действующая
747030,747030,Деятельность в области права,2012-08-15T00:00:00.000Z,2006-08-09T00:00:00.000Z,NaN,Ликвидирована,НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ МЕЖРЕГИОНАЛЬНЫЙ ФОН...,False,0,0.0,...,NaN,Фонд,2006-08-09T00:00:00.000Z,66,Свердловская область,NaN,NaN,NaN,"НФ МФ ""СОДЕЙСТВИЯ БОРЬБЕ С ОРГАНИЗОВАННОЙ ПРЕС...",Действующая
747031,747031,NaN,2006-10-11T00:00:00.000Z,2006-10-11T00:00:00.000Z,1998-01-01T00:00:00.000Z,Ликвидирована,"СОВЕТ САМОУПРАВЛЕНИЯ МИКРОРАЙОНА ""А""",False,0,0.0,...,NaN,Территориальное общественное самоуправление,1998-01-01T00:00:00.000Z,86,Ханты-Мансийский автономный округ - Югра,NaN,NaN,NaN,"СОВЕТ САМОУПРАВЛЕНИЯ МИКРОРАЙОНА ""А""",Действующая
747032,747032,NaN,2006-06-05T00:00:00.000Z,2006-06-05T00:00:00.000Z,1987-12-05T00:00:00.000Z,Ликвидирована,КООПЕРАТИВ РИТУАЛ ПО ОКАЗАНИЮ РИТУАЛЬНЫХ УСЛУГ...,False,0,0.0,...,NaN,Потребительский кооператив,1987-12-05T00:00:00.000Z,42,Кемеровская область,NaN,NaN,NaN,NaN,Действующая


# Предварителньный анализ

Регион регистрации

In [23]:
df_clean['regionName'].value_counts()

regionName
Москва                          80178
Московская область              37927
Санкт-Петербург                 31767
Краснодарский край              23337
Свердловская область            23030
                                ...  
Республика Ингушетия             1249
Еврейская автономная область     1197
Чукотский автономный округ        505
Ненецкий автономный округ         398
Не определено                      91
Name: count, Length: 86, dtype: int64

Логично, что больше всего НКО в самых населённых регионах

In [ ]:
region_group = df_clean.groupby('regionName')['']